In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
def load_groundtruth_pred(ground_truth_filename=None, prediction_filename=None, subset='validation', verbose=False, top_k=1):
    if not ground_truth_filename:
        raise IOError('Please input a valid ground truth file.')
    if not prediction_filename:
        raise IOError('Please input a valid prediction file.')

    ap = None
    hit_at_k = None
    # Import ground truth and predictions.
    ground_truth, activity_index = _import_ground_truth(ground_truth_filename)
    prediction = _import_prediction(prediction_filename, activity_index)

    if verbose:
        print('[INIT] Loaded annotations from {} subset.'.format(subset))
        nr_gt = len(ground_truth)
        print('\tNumber of ground truth instances: {}'.format(nr_gt))
        nr_pred = len(prediction)
        print('\tNumber of predictions: {}'.format(nr_pred))
        
    return ground_truth, prediction

In [3]:
def _import_ground_truth(ground_truth_filename):
    """Reads ground truth file, checks if it is well formatted, and returns
       the ground truth instances and the activity classes.
    Parameters
    ----------
    ground_truth_filename : str
        Full path to the ground truth json file.
    Outputs
    -------
    ground_truth : df
        Data frame containing the ground truth instances.
    activity_index : dict
        Dictionary containing class index.
    """
    with open(ground_truth_filename, 'r') as fobj:
        data = json.load(fobj)
    # Checking format
    # if not all([field in data.keys() for field in self.gt_fields]):
        # raise IOError('Please input a valid ground truth file.')

    # Initialize data frame
    activity_index, cidx = {}, 0
    video_lst, label_lst = [], []
    for videoid, v in data['database'].items():
        if 'validation' != v['subset']:
            continue
        this_label = v['annotations']['label']
        if this_label not in activity_index:
            activity_index[this_label] = cidx
            cidx += 1
        video_lst.append(videoid)
        label_lst.append(activity_index[this_label])
    ground_truth = pd.DataFrame({'video-id': video_lst,
                                 'label': label_lst})
    ground_truth = ground_truth.drop_duplicates().reset_index(drop=True)
    return ground_truth, activity_index

In [4]:
def _import_prediction(prediction_filename, activity_index):
    """Reads prediction file, checks if it is well formatted, and returns
       the prediction instances.
    Parameters
    ----------
    prediction_filename : str
        Full path to the prediction json file.
    Outputs
    -------
    prediction : df
        Data frame containing the prediction instances.
    """
    with open(prediction_filename, 'r') as fobj:
        data = json.load(fobj)
    # Checking format...
    # if not all([field in data.keys() for field in self.pred_fields]):
        # raise IOError('Please input a valid prediction file.')

    # Initialize data frame
    video_lst, label_lst, score_lst = [], [], []
    for videoid, v in data['results'].items():
        for result in v:
            label = activity_index[result['label']]
            video_lst.append(videoid)
            label_lst.append(label)
            score_lst.append(result['score'])
    prediction = pd.DataFrame({'video-id': video_lst,
                               'label': label_lst,
                               'score': score_lst})
    return prediction

In [5]:
ground_truth_filename = "/home/shared/workspace/Resnet3D/3D-ResNets-PyTorch/our_data/ntutest.json"

In [6]:
prediction_filename = "/home/shared/workspace/human-activity-recognition/Efficient-3DCNNs/data/results/resnet_101_50_ccrop/val.json"

In [7]:
top_k = 1

In [8]:
ground_truth, prediction = load_groundtruth_pred(ground_truth_filename, prediction_filename, 'validation', True, top_k)

[INIT] Loaded annotations from validation subset.
	Number of ground truth instances: 18
	Number of predictions: 54


In [9]:
ground_truth

,video-id,label
0,S001C001P001R001A041_rgb,0
1,S001C001P002R001A041_rgb,0
2,S001C001P002R001A042_rgb,1
3,S001C001P001R001A042_rgb,1
4,S001C001P001R001A043_rgb,2
5,S001C001P002R001A043_rgb,2
6,S001C001P001R001A044_rgb,3
7,S001C001P002R001A044_rgb,3
8,S001C001P002R001A045_rgb,4
9,S001C001P001R001A045_rgb,4


In [10]:
prediction

,video-id,label,score
0,S001C001P001R001A041_rgb,5,0.734390
1,S001C001P001R001A041_rgb,6,0.076335
2,S001C001P001R001A041_rgb,4,0.071047
3,S001C001P002R001A041_rgb,5,0.480826
4,S001C001P002R001A041_rgb,6,0.178359
5,S001C001P002R001A041_rgb,4,0.093173
6,S001C001P002R001A042_rgb,6,0.248687
7,S001C001P002R001A042_rgb,5,0.225362
8,S001C001P002R001A042_rgb,3,0.203735
9,S001C001P001R001A042_rgb,3,0.269759


In [11]:
truth_pred = ground_truth

In [12]:
truth_pred

,video-id,label
0,S001C001P001R001A041_rgb,0
1,S001C001P002R001A041_rgb,0
2,S001C001P002R001A042_rgb,1
3,S001C001P001R001A042_rgb,1
4,S001C001P001R001A043_rgb,2
5,S001C001P002R001A043_rgb,2
6,S001C001P001R001A044_rgb,3
7,S001C001P002R001A044_rgb,3
8,S001C001P002R001A045_rgb,4
9,S001C001P001R001A045_rgb,4


In [13]:
def compute_video_hit_at_k(ground_truth, prediction, top_k=1):
    """Compute accuracy at k prediction between ground truth and
    predictions data frames. This code is greatly inspired by evaluation
    performed in Karpathy et al. CVPR14.
    Parameters
    ----------
    ground_truth : df
        Data frame containing the ground truth instances.
        Required fields: ['video-id', 'label']
    prediction : df
        Data frame containing the prediction instances.
        Required fields: ['video-id, 'label', 'score']
    Outputs
    -------
    acc : float
        Top k accuracy score.
    """
    video_ids = np.unique(ground_truth['video-id'].values)
    avg_hits_per_vid = np.zeros(video_ids.size)
    video_name, y_true,y_pred = [],[],[]
    for i, vid in enumerate(video_ids):
        pred_idx = prediction['video-id'] == vid
        if not pred_idx.any():
            continue
        this_pred = prediction.loc[pred_idx].reset_index(drop=True)
        # Get top K predictions sorted by decreasing score.
        sort_idx = this_pred['score'].values.argsort()[::-1][:top_k]
        this_pred = this_pred.loc[sort_idx].reset_index(drop=True)
        # Get labels and compare against ground truth.
        pred_label = this_pred['label'].tolist()
        gt_idx = ground_truth['video-id'] == vid
        gt_label = ground_truth.loc[gt_idx]['label'].tolist()
        avg_hits_per_vid[i] = np.mean([1 if this_label in pred_label else 0
                                       for this_label in gt_label])
        y_pred.append(pred_label)
        y_true.append(gt_label)
        video_name.append(vid)
    return float(avg_hits_per_vid.mean()), y_pred, y_true, video_name

In [14]:
hit_at_k, y_pred, y_true, video_name = compute_video_hit_at_k(ground_truth, prediction, 2)

In [15]:
hit_at_k

0.2222222222222222

In [16]:
video_name

['S001C001P001R001A041_rgb',
 'S001C001P001R001A042_rgb',
 'S001C001P001R001A043_rgb',
 'S001C001P001R001A044_rgb',
 'S001C001P001R001A045_rgb',
 'S001C001P001R001A046_rgb',
 'S001C001P001R001A047_rgb',
 'S001C001P001R001A048_rgb',
 'S001C001P001R001A049_rgb',
 'S001C001P002R001A041_rgb',
 'S001C001P002R001A042_rgb',
 'S001C001P002R001A043_rgb',
 'S001C001P002R001A044_rgb',
 'S001C001P002R001A045_rgb',
 'S001C001P002R001A046_rgb',
 'S001C001P002R001A047_rgb',
 'S001C001P002R001A048_rgb',
 'S001C001P002R001A049_rgb']

In [17]:
y_pred = np.squeeze(y_pred)

In [18]:
y_true = np.squeeze(y_true)

In [19]:
y_pred

array([[5, 6],
       [3, 5],
       [4, 5],
       [5, 4],
       [5, 6],
       [5, 6],
       [5, 6],
       [5, 6],
       [5, 4],
       [5, 6],
       [6, 5],
       [4, 5],
       [5, 6],
       [5, 6],
       [5, 4],
       [5, 6],
       [4, 5],
       [5, 4]])

In [20]:
y_true

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8])

In [21]:
y_pred[17,1]

4

In [22]:
y = np.zeros((5,9))
y[0,:] = y_true[0:9]
y[1,:] = y_pred[0:9,0]
y[2,:] = y_pred[0:9,1]
y[3,:] = y_pred[9:18,0]
y[4,:] = y_pred[9:18,1]

In [23]:
y.shape

(5, 9)

In [24]:
ACTION_NAMES = ["sneezeCough", "staggering", "fallingDown",
                "headache", "chestPain", "backPain",
                "neckPain", "nauseaVomiting", "fanSelf"]

In [25]:
label_y = []
for i in range(y.shape[0]):
    for j in range(y.shape[1]):
        label_y.append(ACTION_NAMES[int(y[i][j])])
label_y = np.reshape(label_y, y.shape)

In [26]:
y

array([[0., 1., 2., 3., 4., 5., 6., 7., 8.],
       [5., 3., 4., 5., 5., 5., 5., 5., 5.],
       [6., 5., 5., 4., 6., 6., 6., 6., 4.],
       [5., 6., 4., 5., 5., 5., 5., 4., 5.],
       [6., 5., 5., 6., 6., 4., 6., 5., 4.]])

In [27]:
label_y

array([['sneezeCough', 'staggering', 'fallingDown', 'headache',
        'chestPain', 'backPain', 'neckPain', 'nauseaVomiting', 'fanSelf'],
       ['backPain', 'headache', 'chestPain', 'backPain', 'backPain',
        'backPain', 'backPain', 'backPain', 'backPain'],
       ['neckPain', 'backPain', 'backPain', 'chestPain', 'neckPain',
        'neckPain', 'neckPain', 'neckPain', 'chestPain'],
       ['backPain', 'neckPain', 'chestPain', 'backPain', 'backPain',
        'backPain', 'backPain', 'chestPain', 'backPain'],
       ['neckPain', 'backPain', 'backPain', 'neckPain', 'neckPain',
        'chestPain', 'neckPain', 'backPain', 'chestPain']], dtype='<U14')

In [28]:
y_df = pd.DataFrame(label_y.T, columns = ['truth', 'Raj_top1', 'Raj_top2', 'Myra_top1', 'Myra_top2'] )

In [29]:
y_df

,truth,Raj_top1,Raj_top2,Myra_top1,Myra_top2
0,sneezeCough,backPain,neckPain,backPain,neckPain
1,staggering,headache,backPain,neckPain,backPain
2,fallingDown,chestPain,backPain,chestPain,backPain
3,headache,backPain,chestPain,backPain,neckPain
4,chestPain,backPain,neckPain,backPain,neckPain
5,backPain,backPain,neckPain,backPain,chestPain
6,neckPain,backPain,neckPain,backPain,neckPain
7,nauseaVomiting,backPain,neckPain,chestPain,backPain
8,fanSelf,backPain,chestPain,backPain,chestPain
